<a href="https://colab.research.google.com/github/fejhekker/5AUA0/blob/master/st_densenet_small_images.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#use the previously made small images
!unzip /content/images
!rm -r images.zip

Archive:  /content/images.zip
   creating: content/images/
   creating: content/images/video_0003/
  inflating: content/images/video_0003/000000.npy  
   creating: content/images/video_0015/
   creating: content/images/video_0025/
  inflating: content/images/video_0025/000000.npy  
   creating: content/images/video_0019/
   creating: content/images/video_0004/
  inflating: content/images/video_0004/0007373.npy  
  inflating: content/images/video_0004/000000.npy  
  inflating: content/images/video_0004/0006565.npy  
  inflating: content/images/video_0004/0005454.npy  
  inflating: content/images/video_0004/0006161.npy  
  inflating: content/images/video_0004/0003838.npy  
  inflating: content/images/video_0004/0005656.npy  
  inflating: content/images/video_0004/0004343.npy  
  inflating: content/images/video_0004/0005252.npy  
  inflating: content/images/video_0004/0006767.npy  
  inflating: content/images/video_0004/0004141.npy  
  inflating: content/images/video_0004/0001111.npy  
  

In [0]:
#load and shuffel the data
import numpy as np
from google.colab import drive
drive.mount('/content/gdrive/')
from sklearn.model_selection import train_test_split

#load the data
sequenceBbox=np.load("gdrive/My Drive/TF 01/sequenceBbox.npy", allow_pickle=True)
sequenceLabel=np.load("gdrive/My Drive/TF 01/sequenceLabel.npy", allow_pickle=True)
sequenceBbox=np.transpose(sequenceBbox)

#shuffel and split the data
sequenceBboxTrain, sequenceBboxTest, sequenceLabelTrain, sequenceLabelTest = train_test_split(
                sequenceBbox, sequenceLabel, test_size=0.33, random_state=42)
nTrain=sequenceLabelTrain.shape[0]
nTest=sequenceLabelTest.shape[0]

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [0]:
#st-densenet 3D
import numpy as np
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, ZeroPadding3D, Dense, Dropout, Activation, Convolution3D, Reshape
from tensorflow.keras.layers import AveragePooling3D, GlobalAveragePooling3D, MaxPooling3D, BatchNormalization

def densenet_model(nb_filter=24, nb_layers = [4,4,4], reduction=0.0, 
                   shape=(16, 100, 100, 3), batch_size=10, growth_rate=2):
    # compute compression factor
    compression = 1.0 - reduction
    nb_dense_block = len(nb_layers)
    
    img_input = Input(shape=shape, name='data')
    
    x = BatchNormalization(name='conv1_bn')(img_input)
    x = Activation('relu', name='relu1')(x)
    x = ZeroPadding3D((3, 3, 3), name='conv1_zeropadding', batch_size=batch_size)(x)
    x = Convolution3D(nb_filter, kernel_size=(7,7,7), strides=(1,2,2), name='conv1', use_bias=False)(x)
    x = ZeroPadding3D((1, 1, 1), name='pool1_zeropadding')(x)
    x = AveragePooling3D((3,3,3),strides=(1,2,2))(x)
    
    stage = 0
    # Add dense blocks
    for block_idx in range(nb_dense_block - 1):
        stage = block_idx+2
        x, nb_filter = dense_block(x, stage, nb_layers[block_idx],nb_filter,growth_rate)

        # Add transition_block
        x = transition_block(x, stage, nb_filter, compression=compression)
        nb_filter = int(nb_filter * compression)

    final_stage = stage + 1
    x, nb_filter = dense_block(x, final_stage, nb_layers[-1], nb_filter,growth_rate)
    
    #x = AveragePooling3D((4,7,7))(x)
    x = GlobalAveragePooling3D()(x)
    x = Dense(1)(x)
    #output = Activation('softmax', name='prob')(x)
    output = Activation('sigmoid')(x)
    #output = Reshape((-1,1))(output)
    #output= tf.keras.backend.squeeze(output,-1)
    return Model(inputs=img_input, outputs=output)

def conv_block(x, stage, branch, nb_filter):
    conv_name_base = 'conv' + str(stage) + '_' + str(branch)
    relu_name_base = 'relu' + str(stage) + '_' + str(branch)

    # 1x1 Convolution (Bottleneck layer)
    inter_channel = nb_filter * 4  
    x = BatchNormalization(name=conv_name_base+'_x1_bn')(x)
    x = Activation('relu', name=relu_name_base+'_x1')(x)
    x = Convolution3D(inter_channel, 1, 1, name=conv_name_base+'_x1', use_bias=False)(x)

    # 3x3 Convolution
    x = BatchNormalization(name=conv_name_base+'_x2_bn')(x)
    x = Activation('relu', name=relu_name_base+'_x2')(x)
    x = ZeroPadding3D((1, 1, 1), name=conv_name_base+'_x2_zeropadding')(x)
    x = Convolution3D(nb_filter, 3, 1, name=conv_name_base+'_x2', use_bias=False)(x)

    return x

def dense_block(x, stage, nb_layers, nb_filter, growth_rate, grow_nb_filters=True):
    concat_feat = x
    for i in range(nb_layers):
        branch = i+1
        x = conv_block(concat_feat, stage, branch, nb_filter)
        concat_feat = tf.concat([concat_feat, x], -1)

        if grow_nb_filters:
            nb_filter += growth_rate
    return concat_feat, nb_filter

def transition_block(x, stage, nb_filter, compression=1.0):

    conv_name_base = 'conv' + str(stage) + '_blk'
    relu_name_base = 'relu' + str(stage) + '_blk'
    pool_name_base = 'pool' + str(stage) 

    x = BatchNormalization(name=conv_name_base+'_bn')(x)
    x = Activation('relu', name=relu_name_base)(x)
    x = Convolution3D(int(nb_filter * compression), 1, 1, name=conv_name_base, use_bias=False)(x)
    x = ZeroPadding3D((0, 1, 1), name=conv_name_base+'_x2_zeropadding')(x)
    x = AveragePooling3D((2, 2, 2), strides=(2, 2, 2), name=pool_name_base)(x)

    return x



In [0]:
#show the model
from tensorflow.keras.utils import plot_model
batch=10
model=densenet_model(nb_filter=24, nb_layers = [4,4,4], reduction=0.0, 
                     shape=(16, 100, 100, 3), batch_size=batch, growth_rate=2)
model.summary()
#plot_model(model, to_file='model.png', show_shapes=True)


Model: "model_24"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               [(None, 16, 100, 100 0                                            
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 100, 100, 12          data[0][0]                       
__________________________________________________________________________________________________
relu1 (Activation)              (None, 16, 100, 100, 0           conv1_bn[0][0]                   
__________________________________________________________________________________________________
conv1_zeropadding (ZeroPadding3 (None, 22, 106, 106, 0           relu1[0][0]                      
___________________________________________________________________________________________

In [0]:
#pipline check +generator
import cv2
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow import keras
%matplotlib inline
import matplotlib.pyplot as plt
import os
import datetime
from google.colab import drive
drive.mount('/content/gdrive/')
%load_ext tensorboard

#set the training variables
batch=10 #ntrain must be devidable by batch
nEpochs=1

def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(a)) #the seed i chosen so that the data stays the same
    return a[p], b[p]

def generator(sequenceBbox,sequenceLabel, batch, mode="train" ):
  count=0
  while True: #keep generating data for the mode.fit
    xBatch=np.array([])
    while xBatch.shape[0]<batch:#get the data per batch
      if count >= sequenceBbox.shape[0]:#when all the data is used begin from the start, but reshuffel
        count=0
        sequenceBbox, sequenceLabel = unison_shuffled_copies(sequenceBbox, sequenceLabel)
      #get the sequence
      bbox= sequenceBbox[count]
      #get the current label and change the dimensions to match the network
      label = sequenceLabel[count]
      x=np.zeros((16,100,100,3))
      features=np.array([])
      for i,image in enumerate(bbox):#for each frame in the sequence
        x[i]=np.load("/content"+image) #load the image along whit the rest of the sequenc
      count+=1 #keep track of the amount of sequences processed
      if xBatch.size == 0:
        xBatch=x
        xBatch=np.expand_dims(xBatch,axis=0)
        yBatch=label
      else:
        x=np.expand_dims(x,axis=0)
        xBatch=np.vstack((xBatch,x))
        yBatch=np.hstack((yBatch,label))
    yield xBatch, yBatch

# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "gdrive/My Drive/TF 01/training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True)

#st-Densenet 3D model
model=densenet_model(nb_filter=24, nb_layers = [4,4,4], reduction=0.0, 
                     shape=(16, 100, 100, 3), batch_size=batch,growth_rate=24)

model.compile(loss=tf.keras.losses.binary_crossentropy, 
              optimizer=tf.keras.optimizers.Adam(lr=0.001),
              metrics=['accuracy',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])

# Save the weights using the `checkpoint_path` format
model.save_weights(checkpoint_path.format(epoch=0))
logdir = os.path.join("logs/4/", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

trainGen=generator(sequenceBboxTrain,sequenceLabelTrain, batch)
testGen=generator(sequenceBboxTest,sequenceLabelTest, batch)
model.fit(trainGen, steps_per_epoch=nTrain//batch,
                    validation_data=testGen, validation_steps=nTest//batch,
                    epochs=nEpochs, callbacks=[tensorboard_callback, cp_callback])

print(datetime.datetime.now())
%tensorboard --logdir logs

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).
The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
52/52 [==============================] - ETA: 0s - loss: 8.0198 - accuracy: 0.5654 - precision_1: 0.6455 - recall_1: 0.6166  
Epoch 00001: saving model to gdrive/My Drive/TF 01/training_2/cp-0001.ckpt
52/52 [==============================] - 3767s 72s/step - loss: 8.0198 - accuracy: 0.5654 - precision_1: 0.6455 - recall_1: 0.6166 - val_loss: 53.9608 - val_accuracy: 0.5885 - val_precision_1: 0.5885 - val_recall_1: 1.0000
2020-06-15 13:45:24.227819


Reusing TensorBoard on port 6006 (pid 4205), started 1:13:51 ago. (Use '!kill 4205' to kill it.)

<IPython.core.display.Javascript object>

In [0]:
#view the results
import sklearn.metrics
size=20
testGen=generator(sequenceBboxTest[:size,:],sequenceLabelTest[:size], size)
pred = model.predict(testGen, batch_size=20, steps=1)
print(pred.shape)
true=np.reshape(sequenceLabelTest[:size],(size,))
print(true.shape)
loss = tf.keras.losses.binary_crossentropy(true, pred)
print(loss)
pred=np.reshape(pred,(size,))
loss = tf.keras.losses.binary_crossentropy(true, pred)
print(loss)
# AP=sklearn.metrics.average_precision_score(true[:size],pred, average=None)
# print("AP ="+str(AP))
# AP=sklearn.metrics.average_precision_score(true[:size],pred, average='micro')
# print("AP ="+str(AP))
# AP=sklearn.metrics.average_precision_score(true[:size],pred, average='macro')
# print("AP ="+str(AP))
# AP=sklearn.metrics.average_precision_score(true[:size],pred, average='weighted')
# print("AP ="+str(AP))


(20, 1)
(20,)
tf.Tensor(
[5.3666334 5.3666334 5.3666334 5.3666334 1.7565914 5.3666334 5.3666334
 4.317544  2.3878121 2.683193  2.8998935 1.7455714 2.1849751 5.3666334
 2.2816396 5.3666334 5.3666334 4.033927  5.3666334 5.3666334], shape=(20,), dtype=float32)
tf.Tensor(3.3103077, shape=(), dtype=float32)


In [0]:
model.evaluate(testGen,nTest)

Instructions for updating:
Please use Model.evaluate, which supports generators.


[267013.9375, 0.574999988079071, 0.574999988079071, 1.0]

In [0]:
#train model from last saved checkpoint
#not fully tested

#st-Densenet 3D model
model=densenet_model(growth_rate=24, nb_filter=64, nb_layers = [4,4,4], reduction=0.9, 
                   dropout_rate=0.0, classes=1, shape=(16, 100, 100, 3), batch_size=batch,
                   with_output_block=True, with_se_layers=False)

# Loads the weights
checkpoint_path = "gdrive/My Drive/TF 01/training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
latest = tf.train.latest_checkpoint(checkpoint_dir)
model.load_weights(latest)

model.compile(loss=keras.losses.binary_crossentropy, 
              optimizer=tf.keras.optimizers.Adam(lr=0.01),
              metrics=['accuracy'])

trainGen=generator(sequenceBboxTrain,sequenceLabelTrain, batch)
testGen=generator(sequenceBboxTest,sequenceLabelTest, batch)
model.fit(trainGen, steps_per_epoch=nTrain//batch,
                    validation_data=testGen, validation_steps=nTest//batch,
                    epochs=nEpochs)